Importar bibliotecas

In [87]:
import pandas as pd
import numpy as np
import html
import nltk
import regex as re
import string
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [4]:
!pip install nltk

^C


In [17]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Importar dados

In [12]:
data = pd.read_csv('G:/Meu Drive/DataScience/DS-Tera/ProjetoBNCC/bncc-classifier/data/raw/bncc_first_classifier.csv')

In [13]:
data_clone = data.copy()

In [14]:
# renomeando as colunas do dataset

data_clone.columns = ['id', 'questoes', 'tipoQuestoes', 'topico', 'slug', 'materia', 'slug.1','etapaEnsino']
data_clone.head()

,id,questoes,tipoQuestoes,topico,slug,materia,slug.1,etapaEnsino
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,historia-da-arte,Arte,arte,Fundamental II
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,algebra-equacoes-do-2o-grau-equacao-do-2o-grau...,Matemática,matematica,Fundamental II
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,outros,Espanhol,espanhol,Fundamental II
3,106855,<p><div> A respeito dos serviços públicos e da...,2,Responsabilidade Civil do Estado,responsabilidade-civil-do-estado,Direito Administrativo,direito-administrativo,Concurso
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,outros,Química,quimica,Fundamental II


## Pre-processamento

* Escrevendo funcões para etapa de pre-processamento de Texto usando regex 

In [67]:
# Função para remover tags de html usando regex

def remove_tags(text: str) -> str:
    pattern = re.compile("<.*?>")
    cleantext = re.sub(pattern, " ", text).replace(u"\xa0", u" ")
    return cleantext

# Função para deixar todo o texto em caixa baixa
def to_lower(text: str) -> str:
    return text.lower()

# Função para remover numeros
def remove_numbers(text: str) -> str:
    pattern = re.compile("[0-9]+")
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover caracteres especiais e pontuação
def remove_punctuation(text: str) -> str:
    pattern = re.compile("[\d\.\,\*\"\'\”\“\?\!\:\-]")
    clean_text = re.sub(pattern," ", text) 
    return clean_text

# Função para remover stopwords em portugues; preposições, conjunções, artigos, etc
def remove_standard_stopwords_pt(text: str) -> str:
    stop_words = set(stopwords.words("portuguese"))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    final_sentence = " ".join(filtered_sentence)
    return final_sentence  

def remove_standard_stopwords_en(text: str) -> str:
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    final_sentence = " ".join(filtered_sentence)
    return final_sentence  
         

In [68]:
# resolve problema de codificação html da coluna 'questoes'
data_clone['questoes_html']= data['question'].astype('str').apply(html.unescape)

# limpando o texto utilizando as funçoes criadas
data_clone['questoes_tags'] = data_clone['questoes_html'].apply(remove_tags)
data_clone['questoes_lower'] = data_clone['questoes_tags'].apply(to_lower)
data_clone['questoes_numbers'] = data_clone['questoes_lower'].apply(remove_numbers)
data_clone['questoes_punct'] = data_clone['questoes_numbers'].apply(remove_punctuation)
data_clone['questoes_stopw_pt'] = data_clone['questoes_punct'].apply(remove_standard_stopwords_pt)
data_clone['questoes_stopw_en'] = data_clone['questoes_stopw_pt'].apply(remove_standard_stopwords_en)


In [69]:
data_clone.head()

,id,questoes,tipoQuestoes,topico,slug,materia,slug.1,etapaEnsino,questoes_html,questoes_tags,questoes_lower,questoes_numbers,questoes_punct,questoes_stopw_pt,questoes_stopw_en
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,historia-da-arte,Arte,arte,Fundamental II,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,algebra-equacoes-do-2o-grau-equacao-do-2o-grau...,Matemática,matematica,Fundamental II,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual...","“achar um número que, somado com 2 seja igual...","“achar um número que, somado com seja igual...",achar um número que somado com seja igual...,achar número somado igual inverso equações rep...,achar número somado igual inverso equações rep...
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,outros,Espanhol,espanhol,Fundamental II,<p>2. Complete os espaços abaixo com os artigo...,2. Complete os espaços abaixo com os artigos ...,2. complete os espaços abaixo com os artigos ...,. complete os espaços abaixo com os artigos ...,complete os espaços abaixo com os artigos ...,complete espaços abaixo artigos determinados i...,complete espaços abaixo artigos determinados i...
3,106855,<p><div> A respeito dos serviços públicos e da...,2,Responsabilidade Civil do Estado,responsabilidade-civil-do-estado,Direito Administrativo,direito-administrativo,Concurso,<p><div> A respeito dos serviços públicos e da...,A respeito dos serviços públicos e da respo...,a respeito dos serviços públicos e da respo...,a respeito dos serviços públicos e da respo...,a respeito dos serviços públicos e da respo...,respeito serviços públicos responsabilidade ci...,respeito serviços públicos responsabilidade ci...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,outros,Química,quimica,Fundamental II,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...


In [70]:
data_clone['questoes_punct']

0         com base em seus conhecimentos sobre o compos...
1          achar um número que  somado com   seja igual...
2            complete os espaços abaixo com os artigos ...
3           a respeito dos serviços públicos e da respo...
4         a chuva ácida é um fenômeno provocado pela po...
                               ...                        
99995     para responder à questão  leia o trecho do co...
99996                         em que pais surgiu o skate  
99997       a teoria comportamental oferece uma varieda...
99998     os machos e fêmeas de mosquitos do gênero  ae...
99999     observe a tabela e veja quantos lanches foram...
Name: questoes_punct, Length: 100000, dtype: object

Verifica a existência de questões para concursos e outros tipos que não é interessante para análise

In [71]:
data_clone['etapaEnsino'].value_counts()

Médio & Pré-Vestibular    35676
Fundamental II            33790
Fundamental I             18379
Concurso                   9875
Militar                    2200
OAB                          80
Name: etapaEnsino, dtype: int64

Selecionando apenas questões relacionadas ao Ensino Fundamental e Médio

In [72]:
data_pre_process = data_clone[data_clone["etapaEnsino"].isin(["Médio & Pré-Vestibular", "Fundamental II", "Fundamental I"])]
data_pre_process.head()

,id,questoes,tipoQuestoes,topico,slug,materia,slug.1,etapaEnsino,questoes_html,questoes_tags,questoes_lower,questoes_numbers,questoes_punct,questoes_stopw_pt,questoes_stopw_en
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,historia-da-arte,Arte,arte,Fundamental II,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,algebra-equacoes-do-2o-grau-equacao-do-2o-grau...,Matemática,matematica,Fundamental II,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual...","“achar um número que, somado com 2 seja igual...","“achar um número que, somado com seja igual...",achar um número que somado com seja igual...,achar número somado igual inverso equações rep...,achar número somado igual inverso equações rep...
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,outros,Espanhol,espanhol,Fundamental II,<p>2. Complete os espaços abaixo com os artigo...,2. Complete os espaços abaixo com os artigos ...,2. complete os espaços abaixo com os artigos ...,. complete os espaços abaixo com os artigos ...,complete os espaços abaixo com os artigos ...,complete espaços abaixo artigos determinados i...,complete espaços abaixo artigos determinados i...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,outros,Química,quimica,Fundamental II,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...
5,2016466,<p><strong>Anexe nesse espa&ccedil;o uma foto ...,4,Proposta,proposta,Redação,redacao,Fundamental I,<p><strong>Anexe nesse espaço uma foto da sua ...,Anexe nesse espaço uma foto da sua ilustraçã...,anexe nesse espaço uma foto da sua ilustraçã...,anexe nesse espaço uma foto da sua ilustraçã...,anexe nesse espaço uma foto da sua ilustraçã...,anexe nesse espaço foto ilustração,anexe nesse espaço foto ilustração


In [73]:
# Verificando se foram selecionadas as etapas targets corretas

data_pre_process['etapaEnsino'].value_counts()

Médio & Pré-Vestibular    35676
Fundamental II            33790
Fundamental I             18379
Name: etapaEnsino, dtype: int64

A variável tipoQuestoes contém items dos tipos:

1. Somatória
2. Múltipla Escolha
3. Certo e Errado
4. Discursiva
5. Redação ou items que é necessário enviar foto da resolução
6. Questões com mais de uma alternativa certa
7. Não representa nada em específico

Iremos remover questoes tipo 5 e 6 pois não se adequam ao tipo de alvo do nosso problema.

In [74]:
data_pre_process["tipoQuestoes"].value_counts(ascending=True)

6        5
7      209
5      999
3     1179
1     2968
4    13881
2    68604
Name: tipoQuestoes, dtype: int64

Obs.: Questoes tipo Multipla escolha e Discursiva somam a maioria das questões

Remover as questoes tipo 5 e 6 pois não se adequam ao nosso problema de Classificação

In [75]:
data_pre_process_quest= data_pre_process[~data_pre_process["tipoQuestoes"].isin([5, 6])]

In [76]:
data_pre_process_quest["tipoQuestoes"].value_counts(ascending=True)

7      209
3     1179
1     2968
4    13881
2    68604
Name: tipoQuestoes, dtype: int64

Existem algumas matérias que não se encaixam na BNCC, por isso será necessário removê-las

In [77]:
data_pre_process_quest["materia"].value_counts(ascending=True)

Língua inglesa              1
Geral                       2
Conhecimentos Gerais        7
Italiano                   11
Francês                    17
Alemão                     25
Ciências Humanas           33
Desenho Geométrico         53
Atualidades                70
Produção de Texto         331
Ensino Religioso          446
Redação                   603
Sociologia               1385
Outros                   1512
Literatura               2192
Espanhol                 2266
Educação Física          2322
Biologia                 3024
Química                  3101
Arte                     3642
Filosofia                3691
Física                   5355
Ciências                 7047
Inglês                   7923
História                 8872
Geografia                9056
Língua Portuguesa       11785
Matemática              12069
Name: materia, dtype: int64

Materias listadas pela BNCC
Ensino Fundamental 1 e 2 :
* Português
* Artes
* Educação Física
* Inglês
* Matemática
* Ciências
* Geografia
* História
* Ensino Religioso

Ensino Médio

* Linguagens e suas Tecnologias (Arte, Educação Física, Língua Inglesa e Língua Portuguesa)
* Matemática e suas Tecnologias (Matemática)
* Ciências da Natureza e suas Tecnologias (Biologia, Física e Química)
* Ciências Humanas e Sociais Aplicadas ( História, Geografia, Filosofia e Sociologia)

Primeiro iremos agrupar materias para coincidir com as listadas na BNCC

In [78]:
data_pre_process_group = data_pre_process_quest.copy()
data_pre_process_group.replace('Língua inglesa','Inglês', inplace = True)
data_pre_process_group.replace('Desenho Geométrico','Matemática', inplace = True)
data_pre_process_group.replace(['Produção de Texto', 'Literatura'],'Língua Portuguesa', inplace = True)

Selecionando apenas as matérias listadas na BNCC para Ensino médio e Fundamental 1 e 2

In [79]:
data_pre_process_group = data_pre_process_group[~data_pre_process_group["materia"].isin([
    'Italiano', 'Espanhol','Francês','Alemão', 'Redação','Atualidades','Geral', 'Conhecimentos Gerais','Outros','Ciências Humanas'])]

In [80]:
data_pre_process_group["materia"].value_counts(ascending=True)

Ensino Religioso       446
Sociologia            1385
Educação Física       2322
Biologia              3024
Química               3101
Arte                  3642
Filosofia             3691
Física                5355
Ciências              7047
Inglês                7924
História              8872
Geografia             9056
Matemática           12122
Língua Portuguesa    14308
Name: materia, dtype: int64

Tokenizaçao das Questoes usando NLTK

In [81]:
data_pre_process_tokens = data_pre_process_group.copy()
data_pre_process_tokens['questoes_tokens'] = data_pre_process_tokens['questoes_stopw_en'].apply(word_tokenize)
data_pre_process_tokens['questoes_tokens'].head()

0    [base, conhecimentos, sobre, compositor, john,...
1    [achar, número, somado, igual, inverso, equaçõ...
4    [chuva, ácida, fenômeno, provocado, poluição, ...
6                        [pagina, nr, livro, didático]
7                                        [chiaroscuro]
Name: questoes_tokens, dtype: object

Stemming - Remove sufixo das palavras e reduz à sua forma basal

In [111]:
# função para aplicar processo de 'stemming' na coluna questoes

stemmer = SnowballStemmer("portuguese")

def stemming_text(text):
    return [stemmer.stem(w) for w in text]

data_pre_process_tokens['questoes_stemm'] = data_pre_process_tokens['questoes_tokens'].apply(stemming_text)

In [112]:
data_pre_process_tokens['questoes_stemm'].head()

0    [bas, conhec, sobr, compositor, john, cag, ana...
1    [achar, númer, som, igual, invers, equaçõ, rep...
4    [chuv, ácid, fenômen, provoc, poluiçã, atmosfé...
6                             [pagin, nr, livr, didát]
7                                         [chiaroscur]
Name: questoes_stemm, dtype: object

Lemmatizing - é o processo de agrupar as diferentes formas flexionadas de uma palavra para que possam ser analisadas como um único item

In [113]:
# função para aplicar processo de 'lemmatizing' na coluna questoes
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]

data_pre_process_tokens['questoes_lemma'] = data_pre_process_tokens['questoes_tokens'].apply(lemmatize_text)

In [114]:
data_pre_process_tokens['questoes_lemma'].head()

0    [base, conhecimentos, sobre, compositor, john,...
1    [achar, número, somado, igual, inverso, equaçõ...
4    [chuva, ácida, fenômeno, provocado, poluição, ...
6                        [pagina, nr, livro, didático]
7                                        [chiaroscuro]
Name: questoes_lemma, dtype: object

Deletando colunas que não utilizaremos para análises futuras

In [ ]:
data_pre_process_tokens.drop(['slug','slug.1'], axis=1, inplace=True)

In [103]:
data_pre_process_tokens.head()

,id,questoes,tipoQuestoes,topico,materia,etapaEnsino,questoes_html,questoes_tags,questoes_lower,questoes_numbers,questoes_punct,questoes_stopw_pt,questoes_stopw_en,questoes_tokens,questoes_stemm,questoes_lemma
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,Arte,Fundamental II,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,com base em seus conhecimentos sobre o compos...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...,"[base, conhecimentos, sobre, compositor, john,...","[bas, conhec, sobr, compositor, john, cag, ana...","[base, conhecimentos, sobre, compositor, john,..."
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,Matemática,Fundamental II,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual...","“achar um número que, somado com 2 seja igual...","“achar um número que, somado com seja igual...",achar um número que somado com seja igual...,achar número somado igual inverso equações rep...,achar número somado igual inverso equações rep...,"[achar, número, somado, igual, inverso, equaçõ...","[achar, númer, som, igual, invers, equaçõ, rep...","[achar, número, somado, igual, inverso, equaçõ..."
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,Química,Fundamental II,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,a chuva ácida é um fenômeno provocado pela po...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...,"[chuva, ácida, fenômeno, provocado, poluição, ...","[chuv, ácid, fenômen, provoc, poluiçã, atmosfé...","[chuva, ácida, fenômeno, provocado, poluição, ..."
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,4,Idade Moderna: Revolução Francesa: Antecedente...,História,Fundamental II,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.,pagina nr. 54 do livro didático.,pagina nr. do livro didático.,pagina nr do livro didático,pagina nr livro didático,pagina nr livro didático,"[pagina, nr, livro, didático]","[pagin, nr, livr, didát]","[pagina, nr, livro, didático]"
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,4,História da Arte: Arte Barroca,Arte,Médio & Pré-Vestibular,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?,o que é chiaroscuro?,o que é chiaroscuro?,o que é chiaroscuro,chiaroscuro,chiaroscuro,[chiaroscuro],[chiaroscur],[chiaroscuro]


In [109]:
data_preprocess_basic = data_pre_process_tokens.copy()
data_preprocess_basic.to_csv('G:/Meu Drive/DataScience/DS-Tera/ProjetoBNCC/bncc-classifier/data/curated/data_preprocess_basic.csv', index = False)